In [1]:
# Thanks to Nicks Niche for the starter code
# in case needed
#!pip install pulp

In [3]:
import pandas as pd
import numpy as np
from pulp import *

from src.utils import fix_name_mismatches
from src.components.import_data import ImportData
from src.components.optimizer import OptimizeLineup

### Set Inputs

In [4]:
week = 12
game_mode = 'classic' # 'showdown' or 'classic'
rank_src = 'FP' # 'adhoc' or 'FP'
matchup = 'ALL'

n_lineups = 25
add_results = False # To review lineups along with actual outcomes

# scratch players
scratches = ['Jayden Reed', 'Dontayvion Wicks']

# Add must haves in lineup
force_ins = ['Jalen Hurts', 'Kyren Williams', 'Travis Kelce', 'Tank Dell']

# week 12
# value plays - jalen hurts, johnathan taylor, tank dell, dalton kincaid
# contrarian - cj stroud, kyren williams, mike evans, travis kelce


### Read in Draft Kings Players/Salaries and Rankings

In [5]:
gd = ImportData(week, game_mode, rank_src, matchup, add_results)
players = gd.read_dk()
player_rankings = gd.read_rankings(players)

In [6]:
# QA check for any large names missing rank data
# May signal a name mismatch across sources
mask = player_rankings['FPPG'].isnull() #& player_rankings['Salary'] > 500
player_rankings[mask].sort_values('Salary', ascending=False).head(5)

,Id,Position,Salary,FPPG,ActualFP
109,Tee Higgins,WR,6200,NaN,NaN
523,Tee Higgins,FLEX,6200,NaN,NaN
557,Dallas Goedert,FLEX,4800,NaN,NaN
22,Jimmy Garoppolo,QB,4800,NaN,NaN
143,Dallas Goedert,TE,4800,NaN,NaN


In [7]:
availables = player_rankings.groupby(['Position', 'Id', 'FPPG', 'Salary']).agg('count')
availables = availables.reset_index()

# screening on availables
#mask = availables['Salary'] > 900
#availables = availables[mask]

availables = availables[~availables['Id'].isin(scratches)].reset_index(drop=True)

#check for any force ins not in list
print(f"Names Not Found: {list(set(force_ins) - set(availables['Id']))}")

Names Not Found: []


### Run Optimizer

In [8]:
op = OptimizeLineup(game_mode, availables, n_lineups, force_ins, add_results)
results = op.run_optimizer()
results.head(15)

,1,2,3,4,5,6,7,8,9,ProjFP
1,DST_Chiefs_,FLEX_Travis_Kelce,QB_Jalen_Hurts,RB_Kyren_Williams,RB_Travis_Etienne_Jr.,TE_Cade_Otton,WR_Justin_Watson,WR_Rondale_Moore,WR_Tank_Dell,119.2
2,DST_Chiefs_,FLEX_Evan_Engram,QB_Jalen_Hurts,RB_Kyren_Williams,RB_Rachaad_White,TE_Travis_Kelce,WR_Justin_Watson,WR_Rondale_Moore,WR_Tank_Dell,118.6
3,DST_Chiefs_,FLEX_Trey_McBride,QB_Jalen_Hurts,RB_Jaylen_Warren,RB_Kyren_Williams,TE_Travis_Kelce,WR_Justin_Watson,WR_Rondale_Moore,WR_Tank_Dell,118.2
4,DST_Chiefs_,FLEX_Travis_Kelce,QB_Jalen_Hurts,RB_Jonathan_Taylor,RB_Kyren_Williams,TE_Cade_Otton,WR_Justin_Watson,WR_Rondale_Moore,WR_Tank_Dell,118.1
5,DST_Chiefs_,FLEX_Trey_McBride,QB_Jalen_Hurts,RB_Jerome_Ford,RB_Kyren_Williams,TE_Travis_Kelce,WR_Justin_Watson,WR_Rondale_Moore,WR_Tank_Dell,118.0
6,DST_Chiefs_,FLEX_Travis_Kelce,QB_Jalen_Hurts,RB_Javonte_Williams,RB_Kyren_Williams,TE_Cade_Otton,WR_Demario_Douglas,WR_Rondale_Moore,WR_Tank_Dell,117.9
7,DST_Chiefs_,FLEX_Travis_Kelce,QB_Jalen_Hurts,RB_Javonte_Williams,RB_Kyren_Williams,TE_Cade_Otton,WR_Diontae_Johnson,WR_Justin_Watson,WR_Tank_Dell,117.8
8,DST_Chiefs_,FLEX_Travis_Kelce,QB_Jalen_Hurts,RB_Derrick_Henry,RB_Kyren_Williams,TE_Cade_Otton,WR_Justin_Watson,WR_Rashid_Shaheed,WR_Tank_Dell,117.7
9,DST_Eagles_,FLEX_Travis_Kelce,QB_Jalen_Hurts,RB_Kyren_Williams,RB_Travis_Etienne_Jr.,TE_Cade_Otton,WR_Justin_Watson,WR_Rondale_Moore,WR_Tank_Dell,117.6
10,DST_Chiefs_,FLEX_Travis_Kelce,QB_Jalen_Hurts,RB_Kyren_Williams,RB_Travis_Etienne_Jr.,TE_Cade_Otton,WR_Jalin_Hyatt,WR_Rondale_Moore,WR_Tank_Dell,117.5


In [9]:
# Current entry is below
results.loc[1,]

1                   DST_Chiefs_
2             FLEX_Travis_Kelce
3                QB_Jalen_Hurts
4             RB_Kyren_Williams
5         RB_Travis_Etienne_Jr.
6                 TE_Cade_Otton
7              WR_Justin_Watson
8              WR_Rondale_Moore
9                  WR_Tank_Dell
ProjFP                    119.2
Name: 1, dtype: object